In [3]:
# check if the GPU is working properly
!nvidia-smi
%rm -rf ./sample_data

Thu Sep 26 09:04:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# Installing dependencies
%pip install numba -q
%pip install engineering-notation -q
%pip install opencv-contrib-python -q
%pip install argcomplete -q
%pip install dv-processing -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 75.1 MB/s eta 0:00:00


In [5]:
# Clone SensorsINI/v2e into colab
!git clone https://github.com/SensorsINI/v2e

Cloning into 'v2e'...
remote: Enumerating objects: 3314, done.
remote: Counting objects: 100% (1084/1084), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 3314 (delta 981), reused 1007 (delta 926), pack-reused 2230 (from 1)
Receiving objects: 100% (3314/3314), 34.34 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (2446/2446), done.


In [6]:
# install v2e
%cd /content/v2e
%pip install .

# SuperSloMo39 will be saved here
!mkdir input

# go back
%cd /content/
%pwd

/content/v2e
Processing /content/v2e
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 10.2 MB/s eta 0:00:00
  Created wheel for v2e: filename=v2e-1.5.1-py3-none-any.whl size=116981 sha256=df581cbfa1e05733bac28a8deb3225707d9b66f9ee325b311604f4e9dddb47ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-to4_yf4u/wheels/77/5f/75/585e5c0362b6f0da630814aee1b3cce77ba29bfd5274e7fa38
Successfully built v2e


In [8]:
# bypass the google large file download confrimation
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx" -O /content/v2e/input/SuperSloMo39.ckpt && rm -rf /tmp/cookies.txt

--2024-09-26 09:05:35--  https://docs.google.com/uc?export=download&confirm=&id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx
Resolving docs.google.com (docs.google.com)... 173.194.202.138, 173.194.202.102, 173.194.202.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.202.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx&export=download [following]
--2024-09-26 09:05:35--  https://drive.usercontent.google.com/download?id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.107.132, 2607:f8b0:400e:c0c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.107.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2429 (2.4K) [text/html]
Saving to: ‘/content/v2e/input/SuperSloMo39.ckpt’

/content/v2e/input/ 100%[===================>]   2

In [9]:
# mount with goole drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
names_file_path = "/content/names.txt" #@param {type:"string"}
input_dir = "/content/drive/MyDrive/V2E/input" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/V2E/output/" #@param {type:"string"}
slowmo_path = "/content/v2e/input/SuperSloMo39.ckpt" #@param {type:"string"}

# Open the file in read mode
with open(names_file_path, "r") as file:
    for name in file:
        name = name.strip()
        path = input_dir + "/" + name 
        output_folder = output_dir + "/" + name

        overwrite = True
        unique_output_folder = True
        out_filename = "events.h5"
        davis_output = True

        skip_video_output = True #@param {type:"boolean"}
        dvs_exposure = "duration .033"
        output_mode = "dvs128" #@param ["dvs128", "dvs240", "dvs346", "dvs640", "dvs1024"]

        input_frame_rate = 15 #@param {type:"number"}
        input_slowmotion_factor =  1

        disable_slomo = True
        timestamp_resolution = 0.001
        auto_timestamp_resolution = True

        slomo_model = slowmo_path

        condition = "Clean" #@param ["Custom", "Clean", "Noisy"]

        #@markdown Do not change following parameters unless you are in the custom mode
        thres = 0.2 #@param {type:"slider", min:0.05, max:1, step:0.01}
        sigma = 0.03 #@param {type:"slider", min:0.01, max:0.25, step:0.001}
        cutoff_hz = 200 #@param {type:"slider", min:0, max:300, step:1}
        leak_rate_hz = 5.18 #@param {type:"slider", min:0, max:100, step:0.01}
        shot_noise_rate_hz = 2.716 #@param {type:"slider", min:0, max:100, step:0.001}


        if condition == "Clean":
            thres = 0.2
            sigma = 0.02
            cutoff_hz = 0
            leak_rate_hz = 0
            shot_noise_rate_hz = 0

        elif condition == "Noisy":
            thres = 0.2
            sigma_thres = 0.05
            cutoff_hz = 30
            leak_rate_hz = 0.1
            shot_noise_rate_hz = 5


        v2e_command = ["v2e"]

        # set the input folder
        # the video_path can be a video file or a folder of images
        v2e_command += ["-i", path]

        # set the output folder
        v2e_command += ["-o", output_folder]

        # if the output will rewrite the previous output
        if overwrite:
            v2e_command.append("--overwrite")

        # if there the output folder is unique
        v2e_command += ["--unique_output_folder", "{}".format(unique_output_folder).lower()]

        # set output configs, for the sake of this tutorial, let's just output HDF5 record
        if davis_output:
            v2e_command += ["--davis_output"]
        v2e_command += ["--dvs_h5", out_filename]
        v2e_command += ["--dvs_aedat2", "None"]
        v2e_command += ["--dvs_text", "None"]

        # in Colab, let's say no preview
        v2e_command += ["--no_preview"]

        # if skip video output
        if skip_video_output:
            v2e_command += ["--skip_video_output"]
        else:
            # set DVS video rendering params
            v2e_command += ["--dvs_exposure", dvs_exposure]

        # set slomo related options
        v2e_command += ["--input_frame_rate", "{}".format(input_frame_rate)]
        v2e_command += ["--input_slowmotion_factor", "{}".format(input_slowmotion_factor)]

        # set slomo data
        if disable_slomo:
            v2e_command += ["--disable_slomo"]
            v2e_command += ["--auto_timestamp_resolution", "false"]
        else:
            v2e_command += ["--slomo_model", slomo_model]
            if auto_timestamp_resolution:
                v2e_command += ["--auto_timestamp_resolution", "{}".format(auto_timestamp_resolution).lower()]
            else:
                v2e_command += ["--timestamp_resolution", "{}".format(timestamp_resolution)]

        # threshold
        v2e_command += ["--pos_thres", "{}".format(thres)]
        v2e_command += ["--neg_thres", "{}".format(thres)]

        # sigma
        v2e_command += ["--sigma_thres", "{}".format(sigma)]

        # DVS non-idealities
        v2e_command += ["--cutoff_hz", "{}".format(cutoff_hz)]
        v2e_command += ["--leak_rate_hz", "{}".format(leak_rate_hz)]
        v2e_command += ["--shot_noise_rate_hz", "{}".format(shot_noise_rate_hz)]

        # append output mode
        v2e_command += [f"--{output_mode}"]

        # Final v2e command

        final_v2e_command = " ".join(v2e_command)

        # print("The Final v2e command:")
        display(final_v2e_command)

        # Run command!
        !$final_v2e_command

'The chosen video file: /content/drive/MyDrive/V2E/input/2.mp4'

The Final v2e command:


'v2e -i /content/drive/MyDrive/V2E/input/2.mp4 -o /content/drive/MyDrive/V2E/output/2 --overwrite --unique_output_folder true --davis_output --dvs_h5 events.h5 --dvs_aedat2 None --dvs_text None --no_preview --dvs_exposure duration .033 --input_frame_rate 15 --input_slowmotion_factor 1 --disable_slomo --auto_timestamp_resolution false --pos_thres 0.2 --neg_thres 0.2 --sigma_thres 0.02 --cutoff_hz 0 --leak_rate_hz 0 --shot_noise_rate_hz 0 --dvs346'

INFO:v2e:torch device is cuda
INFO:v2e:No module named 'gooey': Gooey GUI builder not available, will use command line arguments.
Install with 'pip install Gooey if you want a no-arg GUI to invoke v2e'. See README
INFO:v2e:name 'Gooey' is not defined: Gooey package GUI not available, using command line arguments. 
You can try to install with "pip install Gooey"
INFO:v2ecore.v2e_utils:using output folder /content/drive/MyDrive/V2E/output/2
INFO:v2e:output_in_place==False so made output_folder=/content/drive/MyDrive/V2E/output/2
INFO:v2ecore.v2e_args:
*** arguments:
auto_timestamp_resolution:	False
avi_frame_rate:	30
batch_size:	8
crop:	None
cs_lambda_pixels:	None
cs_tau_p_ms:	None
cutoff_hz:	0.0
ddd_output:	False
disable_slomo:	True
dvs1024:	False
dvs128:	False
dvs240:	False
dvs346:	True
dvs640:	False
dvs_aedat2:	None
dvs_aedat4:	None
dvs_emulator_seed:	0
dvs_exposure:	['duration', '.033']
dvs_h5:	events.h5
dvs_params:	None
dvs_text:	None
dvs_vid:	dvs-video.avi
dvs_vid_full_scale:	2
hdr

'The chosen video file: /content/drive/MyDrive/V2E/input/21.mp4'

...

INFO:v2e:torch device is cuda
INFO:v2e:No module named 'gooey': Gooey GUI builder not available, will use command line arguments.
Install with 'pip install Gooey if you want a no-arg GUI to invoke v2e'. See README
INFO:v2e:name 'Gooey' is not defined: Gooey package GUI not available, using command line arguments. 
You can try to install with "pip install Gooey"
INFO:v2ecore.v2e_utils:using output folder /content/drive/MyDrive/V2E/output/871
INFO:v2e:output_in_place==False so made output_folder=/content/drive/MyDrive/V2E/output/871
INFO:v2ecore.v2e_args:
*** arguments:
auto_timestamp_resolution:	False
avi_frame_rate:	30
batch_size:	8
crop:	None
cs_lambda_pixels:	None
cs_tau_p_ms:	None
cutoff_hz:	0.0
ddd_output:	False
disable_slomo:	True
dvs1024:	False
dvs128:	False
dvs240:	False
dvs346:	True
dvs640:	False
dvs_aedat2:	None
dvs_aedat4:	None
dvs_emulator_seed:	0
dvs_exposure:	['duration', '.033']
dvs_h5:	events.h5
dvs_params:	None
dvs_text:	None
dvs_vid:	dvs-video.avi
dvs_vid_full_scale:	2

'The chosen video file: /content/drive/MyDrive/V2E/input/879.mp4'

The Final v2e command:


'v2e -i /content/drive/MyDrive/V2E/input/879.mp4 -o /content/drive/MyDrive/V2E/output/879 --overwrite --unique_output_folder true --davis_output --dvs_h5 events.h5 --dvs_aedat2 None --dvs_text None --no_preview --dvs_exposure duration .033 --input_frame_rate 15 --input_slowmotion_factor 1 --disable_slomo --auto_timestamp_resolution false --pos_thres 0.2 --neg_thres 0.2 --sigma_thres 0.02 --cutoff_hz 0 --leak_rate_hz 0 --shot_noise_rate_hz 0 --dvs346'

INFO:v2e:torch device is cuda
INFO:v2e:No module named 'gooey': Gooey GUI builder not available, will use command line arguments.
Install with 'pip install Gooey if you want a no-arg GUI to invoke v2e'. See README
INFO:v2e:name 'Gooey' is not defined: Gooey package GUI not available, using command line arguments. 
You can try to install with "pip install Gooey"
INFO:v2ecore.v2e_utils:using output folder /content/drive/MyDrive/V2E/output/879
INFO:v2e:output_in_place==False so made output_folder=/content/drive/MyDrive/V2E/output/879
INFO:v2ecore.v2e_args:
*** arguments:
auto_timestamp_resolution:	False
avi_frame_rate:	30
batch_size:	8
crop:	None
cs_lambda_pixels:	None
cs_tau_p_ms:	None
cutoff_hz:	0.0
ddd_output:	False
disable_slomo:	True
dvs1024:	False
dvs128:	False
dvs240:	False
dvs346:	True
dvs640:	False
dvs_aedat2:	None
dvs_aedat4:	None
dvs_emulator_seed:	0
dvs_exposure:	['duration', '.033']
dvs_h5:	events.h5
dvs_params:	None
dvs_text:	None
dvs_vid:	dvs-video.avi
dvs_vid_full_scale:	2